In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=19)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [3]:
from cluster_model import find_max_weight

find_max_weight(decomp.lattice_scores, K=20)

Most negative absolute value: 1e-06
We have 120 clusters
GOT PATHS [(3465002, [0, 8, 57, 89, 120])]


(None, <networkx.classes.digraph.DiGraph at 0x7a3b001c5a00>)

In [4]:
import utils
importlib.reload(utils)
utils.top_k_dag_paths_dynamic(decomp.lattice_scores, k=200_000)

[([7, 16, 8, 0], 3465),
 ([1, 3, 36, 0], 3135),
 ([29, 20, 27, 0], 2840),
 ([38, 17, 21, 0], 2634),
 ([1, 3, 25, 0], 2538),
 ([5, 4, 6, 0], 2530),
 ([24, 25, 28, 0], 2496),
 ([32, 7, 20, 0], 2318),
 ([18, 11, 5, 0], 2248),
 ([24, 25, 22, 0], 2128),
 ([29, 20, 23, 0], 2070),
 ([1, 3, 10, 0], 2063),
 ([24, 25, 3, 0], 1997),
 ([13, 10, 1, 0], 1984),
 ([1, 3, 23, 0], 1981),
 ([1, 3, 38, 0], 1931),
 ([1, 3, 29, 0], 1902),
 ([11, 33, 5, 0], 1896),
 ([1, 3, 35, 0], 1888),
 ([29, 20, 38, 0], 1874),
 ([1, 3, 21, 0], 1872),
 ([1, 3, 27, 0], 1871),
 ([1, 3, 16, 0], 1870),
 ([7, 16, 29, 0], 1867),
 ([30, 22, 4, 0], 1865),
 ([1, 3, 32, 0], 1863),
 ([1, 3, 11, 0], 1861),
 ([1, 3, 24, 0], 1860),
 ([1, 3, 20, 0], 1859),
 ([1, 3, 17, 0], 1856),
 ([1, 3, 4, 0], 1853),
 ([1, 3, 14, 0], 1853),
 ([1, 3, 18, 0], 1853),
 ([1, 3, 33, 0], 1853),
 ([1, 3, 7, 0], 1852),
 ([1, 3, 22, 0], 1852),
 ([1, 3, 19, 0], 1851),
 ([1, 3, 13, 0], 1850),
 ([1, 3, 0, 0], 1849),
 ([1, 3, 1, 0], 1849),
 ([1, 3, 2, 0], 1849),
 ([

In [22]:
from circuitsvis.utils.render import RenderedHTML, render

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
score_path = [1, 3, 25]

to_score = [d[:100] for d in ds][:100]
scores = decomp.score(
    to_score,
    score_path=score_path
)

DS (3, 2009, 512) 2009
(2009, 3, 512) [38, 17, 21]


In [23]:
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!

# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=[[[[tok]] for tok in s] for s in scores_reord],
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)

In [24]:
scores_per_token_set, scores_per_token_set[top_args]

(array([3.00000000e+000, 0.00000000e+000, 1.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 2.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 3.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        2.00000000e+000, 3.00000000e+000, 3.00000000e+000, 3.00000000e+000,
        2.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 3.00000000e+000,
        2.00000000e+000, 0.00000000e+000, 0.00000000e+000, 3.00000000e+000,
        3.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 2.00000000e+000,
        3.00000000e+000, 3.00000000e+000, 1.00000000e+000, 0.00000000e+000,
        1.00000000e+000, 0.00000000e+000, 0.00000000e+000, 3.00000000e+000,
        0.00000000e+000, 3.00000000e+000, 3.00000000e+000, 1.00000000e+000,
        3.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.00

In [25]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_211101/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
